# **Sentiment Analysis**

Pre Scriptum: Т.к. я не успела сделать первый кейс, предлагаю вам посмотреть на решение похоже задачи, реализованное мной для отбора на Уральскую Проектную Смену в октябре 2019 года. Спасибо.

**Необходимые библиотеки:**
* skitilearn
* xgboost
* codecs
* eli5
* textbolb
* numpy
* pandas
* ntlk
* re
* codecs

In [0]:
!pip install xgboost 
!pip install eli5
!pip install textblob
from sklearn.feature_extraction.text import CountVectorizer
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from textblob import TextBlob
from textblob import Word
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import eli5
import numpy as np
import pandas as pd
import codecs
import re 

# Шаг 1: Работа с данными


Загрузим тренировочные и тестовые выборки из датасета yelp, а также набор **'stopwords'** - слова в языке, не несущие смысловую (только грамматическую) нагрузку. 

*Например: this, is, therefore, etc.*


In [0]:
# train data
train_input_labels = codecs.open(r'/content/yelp/sentiment.train.labels', 'r')
train_input_text = codecs.open(r'/content/yelp/sentiment.train.text', 'r')

# test data
test_input_labels = codecs.open(r'/content/yelp/sentiment.test.labels', 'r')
test_input_text = codecs.open(r'/content/yelp/sentiment.test.text', 'r')

# stop words corpus
stop = set(stopwords.words('english')) 

Так как изначально данные представленны в виде файла без определённого формата, требуется представить их в виде массива. 

Для оптимизации одновремено с этим будем производить работу с данными, а именно:
1.   Приведём все слова в lower case
2.   Избавимся от пунктуации
3.   Лемматизируем, т.е. приведём к словарной (по возможности - начальной) форме каждое слово





In [0]:
# simple function to clean and cast data to list
def get_list(input):
    output = []
    snow = nltk.stem.SnowballStemmer('english')
    for sentence in input:
        sentence = sentence.lower()  
        sentence.replace("\n", "")              
        cleanr = re.compile('<.*?>')
        sentence = re.sub(cleanr, ' ', sentence)     
        sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
        sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)      
    
        words = [Word(word).lemmatize() for word in sentence.split()]
        output.append(" ".join(words))

    return output


# init train and test datasets
train_x = get_list(train_input_text)
train_y = get_list(train_input_labels)

test_x = get_list(test_input_text)
test_y = get_list(test_input_labels)

# Шаг 2: Feature engineering

Следующий шаг - создание представления данных. 

Для решения поставленной задачи будем использовать **"Мешок слов"** как достаточно высокоуровневый метод представления данных. 

В этом случае предложение будет представлено как вектор длины **n**, где **n** - количество уникальных слов во всей выборке (длина условного словаря).
Каждое слово будет представлено индексом - числом вхождений его в предложение.  

In [0]:
# feature engineering: count vectors
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_x)
train_x_count =  count_vect.transform(train_x)
test_x_count =  count_vect.transform(test_x)


# Шаг 3: Классификация

Наконец, приступаем к классификации. 

Самые популярные методы решения задачи о бинарной классификации:
* Логистическая регрессия
* Наивный Байесовский алгоритм
* Дерево решений и случайный лес
* Бустинг
* Метод опорных векторов
* Метод К-ближайших соседей

Последние два метода слишком неэффективны, в то же время мы имеем достаточно большую тренировочную выборку, поэтому опустим их.

Итак, произведём обучение четырёх названных нами моделей. Вынос в отдельный метод сделан для устранения повторяющегося кода.

In [0]:
def train_model(classifier, feature_vector_train, train_y, feature_vector_test, test_y):
    classifier.fit(feature_vector_train, train_y)
    predictions = classifier.predict(feature_vector_test)

    models.append(type(classifier).__name__)
    res = get_metrics(predictions, test_y)
    accuracy.append(res[0])
    precision.append(res[1])
    recall.append(res[2])
    f1.append(res[3])


# init metrics datasets
accuracy, precision, recall, f1 = [], [], [], []
models, labels = [], ['Negative', 'Positive']

# init models
LG_model = LogisticRegression(class_weight='balanced', solver='lbfgs', multi_class='multinomial', random_state=42,  n_jobs = -1)
XGB_model = XGBClassifier(max_depth=10, learning_rate=0.1, n_estimators=100, nthread=50)
RF_model = RandomForestClassifier(n_estimators=100, max_depth=42, random_state=42)
NB_model = MultinomialNB()

# train each model
train_model(LG_model, train_x_count, train_y, test_x_count, test_y)
train_model(NB_model, train_x_count, train_y, test_x_count, test_y)
train_model(XGB_model, train_x_count.tocsc(), train_y, test_x_count.tocsc(), test_y)
train_model(RF_model, train_x_count, train_y, test_x_count, test_y)

После обучения каждой модели будем считать следующие метрики:
* Accuracy
* Precision
* F1
* ReCall

In [0]:
def get_metrics(y_predicted, y_test):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None, average='weighted')  
    
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    
    return accuracy, precision, recall, f1

И выведем полученные данные в виде одной таблицы.




In [0]:
# show results
df_results = pd.DataFrame({'Model': models, 'Accuracy': accuracy, 'Precision': precision, 'ReCall': recall, 'F1': f1})
df_results.head()

<figure>
<center>
<img src='https://drive.google.com/uc?id=1p0VQZxzKTNRgkUVrafJ8gEfHy9a3hTBK' />
<figcaption></figcaption></center>
</figure>

# Шаг 4: Интрепретация

По итогам обучения самую высокую точность показала модель **Логистическая Регрессия - 0.96**. Её и будем интерпретировать. 

Для этого используем библиотеку eli5. Метод show_weights() наглядно представляет, какие слова оказали наибольшее влияние при классификации.

In [0]:
# interpret the best model
eli5.show_weights(LG_model, vec=count_vect, target_names = labels)

<figure>
<center>
<img src='https://drive.google.com/uc?id=1VXNQI_4LZqtBluydY3WL3dHmfy8KMogc' />
<figcaption></figcaption></center>
</figure>